(Summary)=
# Summary

This lesson illustrates how we can ''open the box'' to make our models more interpretable. 

We discussed two classes of explainers: 
- global methods (covered in sections [Feature importance - explainability in one line](Feature_importance_-_explainability_in_one_line.ipynb) and [Global Interpretability](Global_Interpretability.ipynb)) allows explaining the behavior of a prediction model, usually per feature.
- Local methods (covered in section [Local interpretability](Local_Interpretability.ipynb)), on their part, allow giving an explanation for each individual prediction.

We also discussed the difference between model-agnostic and model-specific explainers. Interpretability offers tools to answer both parts of the problem: what and why. Interpretability also provides helpful tools for debugging, increasing model acceptance, and providing justifications during audits. 

However, interpretability can have some drawbacks. 
- Considering interpretability often comes with a drop in predictive performance {cite}`molnar2019`: only answering the what question can be easier than answering both what and why.
- Also, Interpretability disclosure can enable fraudsters (in general) to manipulate the system. Fraudsters could use this information / model instight to their advantage.
- The last limitation is that there is still no consensus on how to assess the quality of the obtained explanations {cite}`molnar2019` and no mathematical definition for interpretability. This lack of formal definition comes from the wide range of questions that arose regarding the usage of machine learning models as decision support systems.

The rest of this section is a summary of the different explainers we considered. We start with a chart listing the top pros and cons for each. Then, we will describe more precisely these pros and cons for each explainer in turn. As it can be seen from the next chart, no explainers are perfect. Therefore, it could be useful to consider a second explainer to confirm an explanation. 


## Take home chart


| Name                                            | Pro | Con |
|:----|:---:|:---:|
| **Global interpretability** |
| Gini feature importance                         | Intuitive, one-line implementation | For trees only, Biaised in favor of high cardinality features |
| Feature permutation                             | Intuitive, one-line implementation | Can be long to compute |
| Partial Dependence Plot                         | Intuitive | Unreliable if correlated features |
| Accumulated Local Effect                        | Unbiased regarding correlated features | Difficult to interpret if strongly correlated |
| **Local interpretability** |
| Individual Conditional Expectation              | Intuitive | Unreliable if correlated features |
| Local Interpretable Model-agnostic Explanations | Human-friendly explanations | Results depends on sampling, Unreliable if correlated features |
| Kernel SHapley Additive exPlanations            | Human-friendly explanations | Slow, Unreliable if correlated features |
| Tree SHapley Additive exPlanations              | Human-friendly explanations, fast for trees | For trees only|

## Feature importance

The Gini feature importance is very intuitive and easy to retrieve. However, it is known to be biased in favor of features with high cardinality {cite}`strobl2007bias`. It can therefore be useful to also consider the permutation feature importance instead.

## Feature permutation

The feature permutation importance is intuitive and easy-to-use. It is affected by the correlated features issue: the permuted predictions are likely to be far from the original samples, polluting the computation of the importance {cite}`hooker2019please`. 

## PDP

PDP represents the average prediction if we force all data points to take a certain value for a feature (or two) of interest. This value is progressively increased to produce the plot. The first limitation is that we can only consider one or two features simultaniously because of the 2D/3D limitation of what we can plot. Another, more important, issue is that PDP assumes independence, which is never the case in fraud detection. Finally, PDP only plots the mean behavior of our dataset. This is why it is often necessary to use ICE on top of PDP.

## ICE

ICE is a more detailed version, local, of PDP. Predictions curves are displayed for each sample. It allows detecting heterogeneities compared to PDP, even if the plot can quickly become overloaded when the number of samples increases. ICE has the same limitation as PDP regarding correlated features.

## ALE

The interpretation of ALE plots is clear, even if it is less intuitive than PDP/ICE. The big advantage of ALE is that it is more robust to correlated variables. For these reasons, ALE is often prefered to PDP/ICE. 

## LIME

First, LIME (as KernelSHAP) works for tabular data, text, and images. The explanations generated by LIME are human-friendly and selective (meaning you can ask for a certain number of explanative features, which can be obtained by playing with the regulation parameter of the LASSO).
The neighborhood-based sampling is a controversy point of LIME for tabular data, and it leads to instability in the explanations. For example, two very close samples can have completely different explanations. Also, sampling is done without any strategy to avoid the nasty correlation effect between features, as described in PDP. 

## KernelSHAP

KernelSHAP improves LIME by proposing the SHAP kernel for its sampling. It ensures one unique ''fair'' solution with three properties: (1) the feature contributions $\phi$'s are additive, (2) absent features get a contribution of 0, and (3) if one feature contribution increases, its $\phi$ also increases. KernelSHAP drawbacks are that it is slower than LIME, and that it has the correlated feature issue, because of the marginal expectation (to evaluate $f(x)$) compromise that was done in its implementation.


## TreeSHAP

To alleviate the time computation problem of KernelSHAP, fast, model-specific, implementations have been developed. It is fast (and can therefore be used to explain all transactions from a dataset) and can be used directly on any tree-based models without requiring any data to train the explainer. Also, it is more robust against the correlated feature issue, as it relies on conditional expectations. Its only drawback is that its usage is restricted to trees.